# ViZDoom Deep RL Ablation Study

**Master's RL Course Final Project**

This notebook runs the complete ViZDoom ablation study on Google Colab with automatic Drive backup.

## Workflow
1. **Setup** - Install dependencies, mount Drive
2. **Quick Test** - Verify environment works (2 min)
3. **Single Training Test** - Confirm learning works (30 min)
4. **Ablation Phase 1** - Algorithm comparison: DQN vs Deep SARSA
5. **Ablation Phase 2** - Learning rate ablation
6. **Ablation Phase 3** - Extensions: DDQN, Dueling, PER
7. **Analysis** - Generate report and plots

**Important**: Run phases in separate sessions if needed (Colab ~12h limit)

---
## 1. Environment Setup

**First time?** Run all cells in Section 1.

**Resuming or updating?** Just run the cell below - it handles everything!

In [ ]:
# ============================================
# ONE-CLICK SETUP / RESUME / UPDATE
# ============================================
# Run this cell to:
# - Install dependencies (first time)
# - Pull latest code (if repo updated)
# - Reconnect to Drive (if session restarted)
# - Resume from where you left off
# ============================================

import sys
import os

IN_COLAB = 'google.colab' in sys.modules
print(f"Running on Colab: {IN_COLAB}")

if IN_COLAB:
    # 1. Install system dependencies (only if needed)
    import shutil
    if not shutil.which('xvfb-run'):
        print("\n[1/5] Installing system dependencies...")
        os.system('apt-get update -qq')
        os.system('apt-get install -qq -y libboost-all-dev libsdl2-dev libopenal-dev xvfb python3-opengl')
    else:
        print("\n[1/5] System dependencies already installed ✓")

    # 2. Install Python packages (only if needed)
    try:
        import vizdoom
        print("[2/5] Python packages already installed ✓")
    except ImportError:
        print("[2/5] Installing Python packages...")
        os.system('pip install -q vizdoom==1.2.4 gymnasium==1.2.3')
        os.system('pip install -q torch torchvision')
        os.system('pip install -q wandb hydra-core omegaconf')
        os.system('pip install -q matplotlib opencv-python numpy')

    # 3. Setup virtual display
    os.system('Xvfb :1 -screen 0 1024x768x24 &')
    os.environ['DISPLAY'] = ':1'
    print("[3/5] Virtual display configured ✓")

    # 4. Mount Google Drive
    from google.colab import drive
    if not os.path.exists('/content/drive/MyDrive'):
        drive.mount('/content/drive')
    print("[4/5] Google Drive mounted ✓")

    # 5. Setup repository and results
    DRIVE_OUTPUT = '/content/drive/MyDrive/vizdoom-ablation-results'
    REPO_PATH = '/content/vizdoom-ablation'
    os.makedirs(DRIVE_OUTPUT, exist_ok=True)

    if os.path.exists(REPO_PATH):
        # Repo exists - pull latest changes
        print("[5/5] Pulling latest code...")
        os.system(f'cd {REPO_PATH} && git pull')
    else:
        # First time - clone repo
        print("[5/5] Cloning repository...")
        os.system(f'git clone https://github.com/lynxrafu/visdoom-ablation.git {REPO_PATH}')

    os.chdir(REPO_PATH)

    # Link results to Drive (recreate symlink)
    if os.path.exists('results') and not os.path.islink('results'):
        os.system('rm -rf results')
    if not os.path.exists('results'):
        os.symlink(DRIVE_OUTPUT, 'results')

    print(f"\n{'='*50}")
    print(f"✓ Ready! Working directory: {os.getcwd()}")
    print(f"✓ Results saved to: {DRIVE_OUTPUT}")

    # Show existing results
    from pathlib import Path
    runs = list(Path(DRIVE_OUTPUT).glob("**/metadata.json"))
    print(f"✓ Existing runs found: {len(runs)}")
    print(f"{'='*50}")

else:
    DRIVE_OUTPUT = 'results'
    os.makedirs(DRIVE_OUTPUT, exist_ok=True)
    print(f"Local mode - results saved to: {DRIVE_OUTPUT}")

In [ ]:
# Skip this cell - handled by ONE-CLICK SETUP above
# (Kept for reference only)

In [ ]:
# Skip this cell - handled by ONE-CLICK SETUP above

In [ ]:
# Skip this cell - handled by ONE-CLICK SETUP above

In [ ]:
# Skip this cell - handled by ONE-CLICK SETUP above

In [ ]:
# Verify imports and login to WandB
import sys
sys.path.insert(0, '.')

import torch
import gymnasium
import vizdoom
import wandb

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Gymnasium: {gymnasium.__version__}")
print(f"ViZDoom: {vizdoom.__version__}")

# Register ViZDoom environments
try:
    import vizdoom.gymnasium_wrapper
except ImportError:
    from vizdoom import gymnasium_wrapper

print("\n✓ All imports successful!")

# Login to WandB (run once per session)
print("\n" + "="*50)
print("WandB Login")
print("="*50)
wandb.login()
print("✓ WandB logged in!")

---
## 2. Quick Test (~2 minutes)

Verify that ViZDoom environment and agent work correctly before long training runs.

In [ ]:
# Test environment
# Register ViZDoom environments with gymnasium
try:
    import vizdoom.gymnasium_wrapper
except ImportError:
    from vizdoom import gymnasium_wrapper

from src.envs import make_vizdoom_env

env = make_vizdoom_env('VizdoomBasic-v0')
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

obs, info = env.reset()
print(f"Initial obs shape: {obs.shape}")

# Take a few random steps
for i in range(5):
    action = env.action_space.sample()
    obs, reward, term, trunc, info = env.step(action)
    print(f"Step {i+1}: action={action}, reward={reward:.2f}")

env.close()
print("\nEnvironment test passed!")

In [ ]:
# Run quick 10-episode training test (WandB disabled for speed)
print("Running quick training test (10 episodes)...")
!python experiments/train.py \
    training.num_episodes=10 \
    logging.wandb_enabled=false \
    logging.csv_log=false

print("\nQuick test passed! Environment and training loop work correctly.")

---
## 3. Single Training Test (~30 minutes)

Train one DQN agent to verify learning happens before running full ablation.

In [ ]:
# Configuration for training runs
# WandB is ENABLED by default for all real training

# Training parameters for test run
TEST_EPISODES = 500  # Enough to see learning
TEST_SCENARIO = 'VizdoomBasic-v0'  # Easiest scenario

print(f"Training config:")
print(f"  Episodes: {TEST_EPISODES}")
print(f"  Scenario: {TEST_SCENARIO}")
print(f"  WandB: Enabled (logging to vizdoom-ablation project)")

In [ ]:
# Train DQN on Basic scenario (with WandB logging)
print(f"Training DQN on {TEST_SCENARIO} for {TEST_EPISODES} episodes...")
print("This takes ~30 minutes. You should see rewards increasing over time.")
print("Track progress at: https://wandb.ai/\n")

!python experiments/train.py \
    env.scenario={TEST_SCENARIO} \
    training.num_episodes={TEST_EPISODES} \
    seed=42

print("\nTraining complete! Check WandB dashboard for detailed metrics.")

In [ ]:
# View the learning curve
import matplotlib.pyplot as plt
from PIL import Image
import glob

# Find the most recent curve
curve_files = glob.glob('results/*_curve.png')
if curve_files:
    latest = max(curve_files, key=os.path.getctime)
    img = Image.open(latest)
    plt.figure(figsize=(12, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Learning Curve')
    plt.show()
else:
    print("No learning curves found yet.")

---
## 4. Ablation Study

Run ablation phases one by one. Each phase saves results to Drive automatically.

| Phase | What it tests | Estimated time |
|-------|--------------|----------------|
| `algorithms` | DQN vs Deep SARSA | ~1-2 hours |
| `lr` | Learning rates (0.0001, 0.001, 0.01) | ~1-2 hours |
| `extensions` | DDQN, Dueling, PER | ~2-3 hours |

**Tip**: If Colab times out, just re-run from where you left off. Results are saved to Drive.

In [ ]:
# Ablation configuration
ABLATION_EPISODES = 500      # Episodes per run
ABLATION_SEEDS = [1, 2, 3]   # Multiple seeds for statistical significance
ABLATION_SCENARIOS = ['VizdoomBasic-v0']  # Start with Basic, add more later

# Convert to string for command line
seeds_str = ' '.join(map(str, ABLATION_SEEDS))
scenarios_str = ' '.join(ABLATION_SCENARIOS)

print(f"Ablation settings:")
print(f"  Episodes: {ABLATION_EPISODES}")
print(f"  Seeds: {ABLATION_SEEDS}")
print(f"  Scenarios: {ABLATION_SCENARIOS}")

In [ ]:
# ============================================
# PHASE 1: Algorithm Comparison (~1-2 hours)
# Compares: DQN (off-policy) vs Deep SARSA (on-policy)
# All runs logged to WandB with unique timestamps
# ============================================
print("=" * 60)
print("PHASE 1: Algorithm Comparison (DQN vs Deep SARSA)")
print("Track progress at: https://wandb.ai/")
print("=" * 60)

!python experiments/ablate.py \
    --phase algorithms \
    --scenarios {scenarios_str} \
    --seeds {seeds_str} \
    --episodes {ABLATION_EPISODES}

print("\nPhase 1 complete! Results saved to Drive and WandB.")

In [ ]:
# ============================================
# PHASE 2: Learning Rate Ablation (~1-2 hours)
# Tests: lr = 0.0001, 0.001, 0.01
# ============================================
print("=" * 60)
print("PHASE 2: Learning Rate Ablation")
print("Track progress at: https://wandb.ai/")
print("=" * 60)

!python experiments/ablate.py \
    --phase lr \
    --scenarios {scenarios_str} \
    --seeds {seeds_str} \
    --episodes {ABLATION_EPISODES}

print("\nPhase 2 complete! Results saved to Drive and WandB.")

In [ ]:
# ============================================
# PHASE 3: DQN Extensions (~2-3 hours)
# Tests: DDQN, Dueling DQN, Prioritized Experience Replay
# ============================================
print("=" * 60)
print("PHASE 3: DQN Extensions (DDQN, Dueling, PER)")
print("Track progress at: https://wandb.ai/")
print("=" * 60)

!python experiments/ablate.py \
    --phase extensions \
    --scenarios {scenarios_str} \
    --seeds {seeds_str} \
    --episodes {ABLATION_EPISODES}

print("\nPhase 3 complete! Results saved to Drive and WandB.")

In [ ]:
# Check results saved on Drive - New organized structure
import glob
import os
from pathlib import Path

print("=" * 60)
print("RESULTS SAVED TO DRIVE")
print("=" * 60)

results_path = Path('results')
if results_path.exists():
    # Find all run directories (contain metadata.json)
    run_dirs = list(results_path.glob("**/metadata.json"))
    print(f"\nTotal training runs: {len(run_dirs)}")

    # Show recent runs
    print("\nRecent runs:")
    for meta_path in sorted(run_dirs, key=lambda x: x.stat().st_mtime, reverse=True)[:5]:
        run_dir = meta_path.parent
        import json
        with open(meta_path) as f:
            meta = json.load(f)
        print(f"  - {run_dir.relative_to(results_path)}")
        print(f"    Agent: {meta.get('agent_type')} | Scenario: {meta.get('scenario')} | Seed: {meta.get('seed')}")

    if IN_COLAB:
        print(f"\nAll results backed up to: {DRIVE_OUTPUT}")
else:
    print("No results yet. Run training first.")

---
## 5. Results Analysis

Analyze all ablation results and generate the IEEE report figures.

**Run this after completing ablation phases** (or after each phase to see intermediate results).

In [ ]:
# Use the ResultsAnalyzer for comprehensive analysis
from src.utils.analysis import ResultsAnalyzer, analyze_results

# Initialize analyzer
analyzer = ResultsAnalyzer("results/")
num_loaded = analyzer.load_all()
print(f"Loaded {num_loaded} experiment results")

In [ ]:
# Display summary table
if num_loaded > 0:
    summary_df = analyzer.summary()
    print("Results Summary:")
    display(summary_df)
else:
    print("No results found. Run ablations first.")

In [ ]:
# Compare algorithms and print rankings
if num_loaded > 0:
    analyzer.compare_algorithms()
    
    for scenario in analyzer.get_scenarios():
        analyzer.print_comparison(scenario)
        
        # Get best algorithm
        best_algo, best_result = analyzer.get_best_algorithm(scenario, "reward")
        print(f"\nBest algorithm for {scenario}: {best_algo} (reward: {best_result.reward_mean:.2f})")

In [ ]:
# Generate complete report with all plots and exports
if num_loaded > 0:
    analyzer.generate_report(
        output_dir="results/report",
        include_plots=True,
        include_tables=True
    )
    
    # Display generated files
    import glob
    report_files = glob.glob("results/report/*")
    print(f"\nGenerated {len(report_files)} report files:")
    for f in report_files:
        print(f"  - {f}")

---
## 6. Export for IEEE Report

Export results in formats suitable for the IEEE report.

---
## Quick Reference

### Your Workflow

| Situation | What to do |
|-----------|------------|
| **First time** | Run cell 2 (ONE-CLICK SETUP) → Run cell 7 (verify imports) → Continue |
| **Session timeout** | Run cell 2 (ONE-CLICK SETUP) → Continue where you left off |
| **Repo updated** | Run cell 2 (ONE-CLICK SETUP) - it auto-pulls changes |
| **Check progress** | Run Section 5 (Analysis) anytime |

### Session Management
- **ONE-CLICK SETUP (cell 2)** handles everything: install, mount, pull, resume
- **Results are on Drive** - never lost even if session dies
- **Just run cell 2** whenever you start/resume - it's smart enough to skip what's already done

### Training Time Estimates
| Phase | Scenarios | Seeds | Episodes | Time |
|-------|-----------|-------|----------|------|
| Quick test | 1 | 1 | 10 | ~2 min |
| Single training | 1 | 1 | 500 | ~30 min |
| Algorithm comparison | 1 | 3 | 500 | ~1-2 hours |
| Learning rate ablation | 1 | 3 | 500 | ~1-2 hours |
| Extensions ablation | 1 | 3 | 500 | ~2-3 hours |

### Output Structure
```
Google Drive/vizdoom-ablation-results/
└── 2025-12-25/
    └── 143052_dqn_VizdoomBasic_v0_lr0.0001_seed42/
        ├── metadata.json      # What was run
        ├── config.yaml        # Full config
        ├── training_log.csv   # All metrics
        ├── summary.json       # Final results
        ├── checkpoints/       # Model weights
        └── plots/             # Learning curves
```

### Tips
- Use **GPU runtime** for faster training
- Run **cell 2 first** every time you open the notebook
- Check **Section 5 (Analysis)** to see all your runs
- Each run gets a **unique timestamp** - nothing is ever overwritten

---
## Notes

### Session Management
- **Colab free tier**: ~12 hour limit per session
- **If session times out**: Re-run cells 1-7 (setup), then continue from where you left off
- **Results are safe**: All outputs saved to Google Drive automatically

### Training Time Estimates
| Phase | Scenarios | Seeds | Episodes | Time |
|-------|-----------|-------|----------|------|
| Quick test | 1 | 1 | 10 | ~2 min |
| Single training | 1 | 1 | 500 | ~30 min |
| Algorithm comparison | 1 | 3 | 500 | ~1-2 hours |
| Learning rate | 1 | 3 | 500 | ~1-2 hours |
| Extensions | 1 | 3 | 500 | ~2-3 hours |

### For Full Ablation (All Scenarios)
To run on all 3 scenarios, update the config cell:
```python
ABLATION_SCENARIOS = ['VizdoomBasic-v0', 'VizdoomTakeCover-v0', 'VizdoomDeathmatch-v0']
```
This will take significantly longer (~10-15 hours total).

### Tips
- Use **GPU runtime** for faster training (Runtime > Change runtime type > GPU)
- Monitor progress via printed episode rewards
- Check Drive periodically to verify results are saving
- Run analysis after each phase to see intermediate results